In [3]:
import yfinance as yf
import pandas as pd
import numpy as np

# Define the ETFs (as provided in Exhibit I)
sector_etfs = ['XLB', 'XLC', 'XLE', 'XLF', 'XLI', 'XLK', 'XLP', 'XLRE', 'XLU', 'XLV', 'XLY']

# Download historical monthly data until 2024-09-01
start_date = "2014-01-01"
end_date = "2024-10-01"
data = yf.download(sector_etfs, start=start_date, end=end_date, interval='1mo')['Adj Close']

# Drop rows with NaN values (if any)
data = data.dropna()

# Ensure datetime index is timezone unaware
data.index = data.index.tz_convert(None)

# Calculate monthly returns
monthly_returns = data.pct_change().dropna()

# Calculate average monthly return and standard deviation for each ETF
average_monthly_return = monthly_returns.mean()
monthly_std_dev = monthly_returns.std()

# Create a DataFrame to hold the statistics
portfolio_stats = pd.DataFrame({
    'Average Monthly Return': average_monthly_return,
    'Monthly Standard Deviation': monthly_std_dev
})

# Calculate portfolio-level metrics
initial_weights = np.array([6300000, 4950000, 2250000, 6300000, 4050000, 5400000, 2250000, 6750000, 1350000, 1800000, 3600000]) / 45000000
portfolio_return = np.dot(initial_weights, portfolio_stats['Average Monthly Return'])
portfolio_variance = np.dot(initial_weights.T, np.dot(monthly_returns.cov(), initial_weights))
portfolio_std_dev = np.sqrt(portfolio_variance)
rf_rate = 0.03 / 12  # Assume risk-free rate is 3% annual, convert to monthly
sharpe_ratio = (portfolio_return - rf_rate) / portfolio_std_dev

# Add portfolio-level metrics to the DataFrame
portfolio_metrics = pd.DataFrame({
    'Metric': ['Expected Monthly Return', 'Monthly Standard Deviation', 'Sharpe Ratio'],
    'Value': [portfolio_return, portfolio_std_dev, sharpe_ratio]
})

# Display the results
print(portfolio_stats)
print("\nPortfolio-Level Metrics:")
print(portfolio_metrics)

# Save to Excel
with pd.ExcelWriter("oct24_data.xlsx") as writer:
    portfolio_stats.to_excel(writer, sheet_name='Sector Statistics', index=True)
    portfolio_metrics.to_excel(writer, sheet_name='Portfolio Metrics', index=False)
    monthly_returns.to_excel(writer, sheet_name='Monthly Returns', index=True)

# Summary
print("\nPortfolio statistics, portfolio-level metrics, and monthly returns have been saved to 'portfolio_statistics.xlsx'")


[*********************100%***********************]  11 of 11 completed

        Average Monthly Return  Monthly Standard Deviation
Ticker                                                    
XLB                   0.010106                    0.062516
XLC                   0.010933                    0.059031
XLE                   0.010936                    0.103529
XLF                   0.010344                    0.065173
XLI                   0.011084                    0.062807
XLK                   0.018648                    0.063997
XLP                   0.008965                    0.040713
XLRE                  0.008543                    0.058336
XLU                   0.009617                    0.049761
XLV                   0.009809                    0.044362
XLY                   0.011060                    0.068343

Portfolio-Level Metrics:
                       Metric     Value
0     Expected Monthly Return  0.011143
1  Monthly Standard Deviation  0.053968
2                Sharpe Ratio  0.160154

Portfolio statistics, portfolio-level metrics,

In [17]:
print(initial_weights)

[0.14 0.11 0.05 0.14 0.09 0.12 0.05 0.15 0.03 0.04 0.08]
